In [54]:
'''
Test hypothesis that if VIX (cost of S&P 500 options), VVIX (cost of VIX options) and S&P 500 all decline,the market moves
higher in the following days. 

'''

# Import libraries

import yfinance as yf # YFinance wrapper to retrieve prices
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
%matplotlib inline

In [55]:
# import data
tickers=['^VIX','^VVIX','^GSPC']
start='2007-01-01' # VVIX prices start in 2007

data=pd.DataFrame()
for t in tickers:
    data[t]=yf.download(t,start=start)['Adj Close'] #onlt need Adjusted Close

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [56]:
# Inspect data
data.head()

,^VIX,^VVIX,^GSPC
Date,,,
2007-01-03,12.04,87.629997,1416.599976
2007-01-04,11.51,88.190002,1418.339966
2007-01-05,12.14,90.169998,1409.709961
2007-01-08,12.00,92.040001,1412.839966
2007-01-09,11.91,92.760002,1412.109985


In [57]:
# Inspect data
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3416 entries, 2007-01-03 to 2020-07-28
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ^VIX    3416 non-null   float64
 1   ^VVIX   3413 non-null   float64
 2   ^GSPC   3416 non-null   float64
dtypes: float64(3)
memory usage: 106.8 KB


In [58]:
# Check null values and adjust
check_nan=data.isnull()
check_nan.loc[check_nan['^VVIX']==True]

,^VIX,^VVIX,^GSPC
Date,,,
2010-11-11,False,True,False
2019-07-05,False,True,False
2020-06-11,False,True,False


In [59]:
#fill null value with forward fill method and rename columns
data['^VVIX'].fillna(method='ffill',inplace=True)
data.rename(columns={'^VIX':'vix','^VVIX':'VVIX','^GSPC':'spx'},inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3416 entries, 2007-01-03 to 2020-07-28
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   vix     3416 non-null   float64
 1   VVIX    3416 non-null   float64
 2   spx     3416 non-null   float64
dtypes: float64(3)
memory usage: 106.8 KB


In [60]:
#Calculate returns for VIX, VVIX, SPX
for col in data.columns:
    data['{}_ret'.format(col)]=data[col].pct_change()
data.head()

,vix,VVIX,spx,vix_ret,VVIX_ret,spx_ret
Date,,,,,,
2007-01-03,12.04,87.629997,1416.599976,NaN,NaN,NaN
2007-01-04,11.51,88.190002,1418.339966,-0.044020,0.006391,0.001228
2007-01-05,12.14,90.169998,1409.709961,0.054735,0.022451,-0.006085
2007-01-08,12.00,92.040001,1412.839966,-0.011532,0.020739,0.002220
2007-01-09,11.91,92.760002,1412.109985,-0.007500,0.007823,-0.000517


In [61]:
# Calculate future returns for spx (split into 2 steps in order to view future prices as well)
# 1. Add spx future price columns using .shift(-period) method

#######################
periods=[5,10,15] # Define forward days (periods)
#######################

for days in periods:
    data['fut_price_{}'.format(days)]=data['spx'].shift(-days)
data.tail()

,vix,VVIX,spx,vix_ret,VVIX_ret,spx_ret,fut_price_5,fut_price_10,fut_price_15
Date,,,,,,,,,
2020-07-22,24.320000,112.839996,3276.020020,-0.020934,-0.022692,0.005747,NaN,NaN,NaN
2020-07-23,26.080000,116.699997,3235.659912,0.072368,0.034208,-0.012320,NaN,NaN,NaN
2020-07-24,25.840000,117.220001,3215.629883,-0.009202,0.004456,-0.006190,NaN,NaN,NaN
2020-07-27,24.740000,112.709999,3239.409912,-0.042570,-0.038475,0.007395,NaN,NaN,NaN
2020-07-28,24.309999,110.800003,3232.510010,-0.017381,-0.016946,-0.002130,NaN,NaN,NaN


In [62]:
# 2. calculate future returns for spx using .pct_change(period) method
for days in periods:
    data['spx_fut_rtn_{}'.format(days)]=data['fut_price_{}'.format(days)].pct_change(days)
data.tail()

,vix,VVIX,spx,vix_ret,VVIX_ret,spx_ret,fut_price_5,fut_price_10,fut_price_15,spx_fut_rtn_5,spx_fut_rtn_10,spx_fut_rtn_15
Date,,,,,,,,,,,,
2020-07-22,24.320000,112.839996,3276.020020,-0.020934,-0.022692,0.005747,NaN,NaN,NaN,-0.013281,-0.013281,-0.013281
2020-07-23,26.080000,116.699997,3235.659912,0.072368,0.034208,-0.012320,NaN,NaN,NaN,-0.000973,-0.000973,-0.000973
2020-07-24,25.840000,117.220001,3215.629883,-0.009202,0.004456,-0.006190,NaN,NaN,NaN,0.005249,0.005249,0.005249
2020-07-27,24.740000,112.709999,3239.409912,-0.042570,-0.038475,0.007395,NaN,NaN,NaN,-0.002130,-0.002130,-0.002130
2020-07-28,24.309999,110.800003,3232.510010,-0.017381,-0.016946,-0.002130,NaN,NaN,NaN,0.000000,0.000000,0.000000


In [63]:
# Drop null value and inspect data
data.dropna(inplace=True) 
summary=data.describe()
summary

,vix,VVIX,spx,vix_ret,VVIX_ret,spx_ret,fut_price_5,fut_price_10,fut_price_15,spx_fut_rtn_5,spx_fut_rtn_10,spx_fut_rtn_15
count,3386.000000,3386.000000,3386.000000,3386.000000,3386.000000,3386.000000,3386.000000,3386.000000,3386.000000,3386.000000,3386.000000,3386.000000
mean,19.810777,90.487094,1837.465153,0.003466,0.001434,0.000318,1840.042238,1842.681535,1845.333611,0.001519,0.002996,0.004513
std,9.850440,15.335140,650.869860,0.082620,0.052401,0.013214,652.691863,654.712185,656.726707,0.025814,0.034839,0.042641
min,9.140000,59.740002,676.530029,-0.295727,-0.210547,-0.119841,676.530029,676.530029,676.530029,-0.183401,-0.258846,-0.283536
25%,13.490000,80.717499,1314.527527,-0.042595,-0.028084,-0.004000,1314.527527,1314.527527,1314.527527,-0.009168,-0.010845,-0.012176
50%,16.790001,88.340000,1721.940002,-0.005961,-0.004045,0.000671,1744.580017,1753.369995,1761.875000,0.003663,0.006599,0.010588
75%,22.647500,97.642502,2343.532471,0.036407,0.024198,0.005717,2348.142456,2351.144958,2356.732544,0.014532,0.021269,0.027146
max,82.690002,207.589996,3386.149902,1.155979,0.570027,0.115800,3386.149902,3386.149902,3386.149902,0.191112,0.216384,0.272039


In [64]:
######################################################
### Hypothesis Testing ####
######################################################

mask_1=data['vix_ret']<0 # VIX return negative
mask_2=data['VVIX_ret']<0 # VVIX return negative
mask_3=data['spx_ret']<0 #SPX return negative

column_list=[]
for period in periods:
    column_list.append('spx_fut_rtn_{}'.format(period))
filtered_df=data[column_list].loc[mask_1&mask_2&mask_3]
filtered_summary=filtered_df.describe()
filtered_summary # Summary dataframe of SPX future returns (periods 1,2,3) given conditions (masks) 1,2,3

,spx_fut_rtn_5,spx_fut_rtn_10,spx_fut_rtn_15
count,208.000000,208.000000,208.000000
mean,0.002583,0.004993,0.008640
std,0.030990,0.037741,0.048285
min,-0.181955,-0.144356,-0.230488
25%,-0.010263,-0.013896,-0.011584
50%,0.004095,0.005708,0.011255
75%,0.016835,0.024717,0.028757
max,0.173974,0.190525,0.272039


In [69]:
def fwd_returns_analysis (df):
    for period in periods:
        wins=df['spx_fut_rtn_{}'.format(period)].loc[df['spx_fut_rtn_{}'.format(period)]>0].count()
        percent_winners=round(wins/df['spx_fut_rtn_{}'.format(period)].count(),2)*100
        max_gain=df['spx_fut_rtn_{}'.format(period)].loc[df['spx_fut_rtn_{}'.format(period)]>0].max()*100
        max_loss=df['spx_fut_rtn_{}'.format(period)].loc[df['spx_fut_rtn_{}'.format(period)]<0].min()*100
        avg_gain=df['spx_fut_rtn_{}'.format(period)].loc[df['spx_fut_rtn_{}'.format(period)]>0].mean()*100
        avg_loss=df['spx_fut_rtn_{}'.format(period)].loc[df['spx_fut_rtn_{}'.format(period)]<0].mean()*100
    
        print('{}-DAYS FORWARD: '.format(period))
        print('Pct winners: '+str(percent_winners)+'%')
        print('Max Gain: '+str(round(max_gain,2))+'%')
        print('Max Loss: '+str(round(max_loss,2))+'%')
        print('Avg Gain: '+str(round(avg_gain,2))+'%')
        print('Avg Loss: '+str(round(avg_loss,2))+'%')
        print('')

In [76]:
print('VIX down, VVIX down, SPX down ('+str(len(filtered_df))+' days)')
fwd_returns_analysis(filtered_df)

VIX down, VVIX down, SPX down (208 days)
5-DAYS FORWARD: 
Pct winners: 59.0%
Max Gain: 17.4%
Max Loss: -18.2%
Avg Gain: 1.91%
Avg Loss: -2.13%

10-DAYS FORWARD: 
Pct winners: 56.99999999999999%
Max Gain: 19.05%
Max Loss: -14.44%
Avg Gain: 2.74%
Avg Loss: -2.44%

15-DAYS FORWARD: 
Pct winners: 65.0%
Max Gain: 27.2%
Max Loss: -23.05%
Avg Gain: 3.13%
Avg Loss: -3.32%



In [78]:
print('All Days ('+str(len(data))+' days)')
fwd_returns_analysis(data)

All Days (3386 days)
5-DAYS FORWARD: 
Pct winners: 59.0%
Max Gain: 19.11%
Max Loss: -18.34%
Avg Gain: 1.62%
Avg Loss: -1.95%

10-DAYS FORWARD: 
Pct winners: 62.0%
Max Gain: 21.64%
Max Loss: -25.88%
Avg Gain: 2.19%
Avg Loss: -2.76%

15-DAYS FORWARD: 
Pct winners: 64.0%
Max Gain: 27.2%
Max Loss: -28.35%
Avg Gain: 2.71%
Avg Loss: -3.49%

